<a href="https://colab.research.google.com/github/mhm930/RedBufferAssignment/blob/master/Darkflow_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

git clone into the darkflow directory on github

cd into the darkflow directory and run "python3 setup.py build_ext --inplace" to install darkflow

And follow the otehr steps to install drakflow for this particular notebook 

Notes : % Before cd and ! before any other command

In [1]:
!git clone https://github.com/thtrieu/darkflow

Cloning into 'darkflow'...
remote: Enumerating objects: 2706, done.
remote: Total 2706 (delta 0), reused 0 (delta 0), pack-reused 2706
Receiving objects: 100% (2706/2706), 32.98 MiB | 9.17 MiB/s, done.
Resolving deltas: 100% (1760/1760), done.


In [2]:
%cd darkflow

/content/darkflow


In [3]:
!python3 setup.py build_ext --inplace

Compiling darkflow/cython_utils/nms.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo2_findboxes.pyx because it changed.
Compiling darkflow/cython_utils/cy_yolo_findboxes.pyx because it changed.
[1/3] Cythonizing darkflow/cython_utils/cy_yolo2_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo2_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/3] Cythonizing darkflow/cython_utils/cy_yolo_findboxes.pyx
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/darkflow/darkflow/cython_utils/cy_yolo_findboxes.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[3/3] Cythonizing darkflow/cython_utils/nms.pyx
/usr/

In [4]:
!pip install .

Processing /content/darkflow
  Created wheel for darkflow: filename=darkflow-1.0.0-cp36-cp36m-linux_x86_64.whl size=831105 sha256=587e552b57380419cf7e591999b3233ac805496cc2764dc189a659ffa281469d
  Stored in directory: /tmp/pip-ephem-wheel-cache-oo5w7yzq/wheels/2f/3a/c5/e84e79d73d5a73aa1b5129a66a40947d9d77a32ebed501e431
Successfully built darkflow


Dark flow successfully installed. Now importing to check the installation - **Training Part**



In [0]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time

option = {
   'model': '/content/darkflow/cfg/tiny-yolo-voc.cfg',
   'load': '/content/gdrive/My Drive/bin/tiny-yolo-voc.weights',
   'threshold': 0.15,
   'gpu': 1.0,
   'annotation' :'/content/gdrive/My Drive/Annotations3xml',
   'dataset' : '/content/gdrive/My Drive/Humanoid3',
   'epoch': 100,
   'train': True,
    'batch' :8
}

tfnet = TFNet(option)
tfnet.train()





Parsing ./cfg/tiny-yolo-voc.cfg
Parsing /content/darkflow/cfg/tiny-yolo-voc.cfg
Loading /content/gdrive/My Drive/bin/tiny-yolo-voc.weights ...
Successfully identified 63471556 bytes
Finished in 0.029654264450073242s
Model has a VOC model name, loading VOC labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | max

In [12]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time

options = {"model": '/content/darkflow/cfg/tiny-yolo-voc.cfg',
           "load": -1,
           "gpu": 1.0}
           
tfnet2 = TFNet(options)
tfnet2.load_from_ckpt()
imgcv = cv2.imread("/content/gdrive/My Drive/training/images/1.jpg")
result = tfnet2.return_predict(imgcv)
print(result)

class_names = ['aeroplane','bicycle','bird','boat']
num_classes = len(class_names)
class_colors = []
for i in range(0, num_classes):
    hue = 255*i/num_classes
    col = np.zeros((1,1,3)).astype("uint8")
    col[0][0][0] = hue
    col[0][0][1] = 128
    col[0][0][2] = 255
    cvcol = cv2.cvtColor(col, cv2.COLOR_HSV2BGR)
    col = (int(cvcol[0][0][0]), int(cvcol[0][0][1]), int(cvcol[0][0][2]))
    class_colors.append(col)
    
    print (class_colors)
    
for item in result:
    tlx = item['topleft']['x']
    tly = item['topleft']['y']
    brx = item['bottomright']['x']
    bry = item['bottomright']['y']
    label = item['label']
    conf = item['confidence']
    print (conf)
    
    if conf > 0.6:
    
        for i in class_names:
            if label == i:
                class_num = class_names.index(i)
                break

    
        cv2.rectangle(imgcv, (tlx, tly), (brx, bry), class_colors[class_num], 2)
    
        text = label + " " + ('%.2f' % conf)
        cv2.rectangle(imgcv, (tlx, tly - 15), (tlx + 100, tly + 5), class_colors[class_num], -1)
        cv2.putText(imgcv, text, (tlx, tly), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
#cv2.imshow("imagecv",imgcv)
#cv2.waitKey()
cv2.imwrite('imagecv.jpg',imgcv)

Parsing /content/darkflow/cfg/tiny-yolo-voc.cfg
Loading None ...
Finished in 0.00018930435180664062s
Model has a VOC model name, loading VOC labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 26, 26, 2

W0820 08:15:15.107660 139956537964416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Loading from ./ckpt/tiny-yolo-voc-100
Finished in 2.1694631576538086s

Loading from ./ckpt/tiny-yolo-voc-100
[{'label': 'bicycle', 'confidence': 0.8824437, 'topleft': {'x': 105, 'y': 90}, 'bottomright': {'x': 244, 'y': 235}}, {'label': 'person', 'confidence': 0.8554902, 'topleft': {'x': 127, 'y': 25}, 'bottomright': {'x': 235, 'y': 178}}]
[(127, 127, 255)]
[(127, 127, 255), (140, 255, 127)]
[(127, 127, 255), (140, 255, 127), (255, 127, 157)]
[(127, 127, 255), (140, 255, 127), (255, 127, 157), (127, 174, 255)]
0.8824437
0.8554902


True

In [0]:
!mkdir ckpt

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive"

 1308964847.672338131.dat
 14-F-MS_CP-09_Final_submission.docx
'3D reconstructon step by step.pdf'
'Annotation Software.rar'
 Answers_to_mail.docx
 App_Maham_Khan.zip
'card data'
'Colab Notebooks'
'Computer vision'
'Copy of Project Description.gdoc'
 Current_GUI.rar
 data_new.7z
 Dataset_17_10_2017.rar
 Dataset.rar
 Dataset_Software.rar
 daum_equation_1399736974316.png
 DemoExecutable+sourcecode.rar
 Demo-Executable.zip
 DemoIMU_GraphicalRepresentation.rar
'Depth Images.zip'
 dist
 DonaldTrumpSpeech.wav
 Electric_Machinery_Fundamentals_4th_Edition.rar
 Executable_Application.rar
 Feature_Calculation.py
 Final.pdf
'GUI Developement (1).rar'
'GUI Developement.rar'
 GUI_updated_2.rar
 GUI_updated_3.rar
 HuDroid-0.1-debug.apk
'Human Recognition Software-0.0.1.rar'
'Human Recognition Software-0.0.2.zip'
 Images_26_3_2018.zip
 Images2Excel-6.0.0.rar
'Images2Excel-7.0.0(IMU) (1).rar'
'Images2Excel-7.0.0(IMU).rar'
'Images2Excel-7.0.0(IMU).zip'
'Left Images.zip'
 Lucidchart
'Maham Khan_applicat